## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Tiksi,RU,71.6872,128.8694,-25.76,99,100,3.78,overcast clouds
1,1,Road Town,VG,18.4167,-64.6167,25.00,83,40,3.60,scattered clouds
2,2,Maniwaki,CA,46.3834,-75.9660,-1.00,24,0,3.09,clear sky
3,3,Kayerkan,RU,69.3497,87.7602,-25.52,87,25,2.69,scattered clouds
4,4,Mataura,NZ,-46.1927,168.8643,13.89,67,100,3.58,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature in Celcius you would like for your trip? "))
max_temp = float(input("What is the maximum temperature in Celcius you would like for your trip? "))

What is the minimum temperature in Celcius you would like for your trip? 28
What is the maximum temperature in Celcius you would like for your trip? 33


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
11,11,New Norfolk,AU,-42.7826,147.0587,29.44,21,0,2.68,clear sky
31,31,Avarua,CK,-21.2078,-159.7750,29.00,74,56,5.66,broken clouds
32,32,Hobart,AU,-42.8794,147.3294,29.44,21,0,8.75,clear sky
40,40,Vaini,TO,-21.2000,-175.2000,29.00,79,75,3.60,broken clouds
43,43,Cam Ranh,VN,11.9214,109.1591,28.00,78,75,1.03,broken clouds
...,...,...,...,...,...,...,...,...,...,...
542,542,Nogales,MX,31.3086,-110.9422,28.33,11,1,4.12,clear sky
573,573,Miri,MY,4.4148,114.0089,28.00,83,20,2.57,few clouds
587,587,Pochutla,MX,15.7432,-96.4661,30.00,54,75,2.09,broken clouds
625,625,Manggar,ID,-2.8833,108.2667,28.54,73,66,3.81,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID        0
City           0
Country        0
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# no empty cells but would drop with the following code
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
11,New Norfolk,AU,29.44,clear sky,-42.7826,147.0587,
31,Avarua,CK,29.00,broken clouds,-21.2078,-159.7750,
32,Hobart,AU,29.44,clear sky,-42.8794,147.3294,
40,Vaini,TO,29.00,broken clouds,-21.2000,-175.2000,
43,Cam Ranh,VN,28.00,broken clouds,11.9214,109.1591,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()  
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except:
        print("Hotel not found... skipping.")   
        
print('Search is done')        

Hotel not found... skipping.
Hotel not found... skipping.
Search is done


In [9]:
# No null values but 2 empty hotels when looking at the dataframe
hotel_df.isnull().sum()

City           0
Country        0
Max Temp       0
Description    0
Lat            0
Lng            0
Hotel Name     0
dtype: int64

In [10]:

hotel_df.tail(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
456,Palu,ID,30.62,overcast clouds,-0.8917,119.8707,Hotel Santika Palu
467,Kosum Phisai,TH,30.56,broken clouds,16.2486,103.0674,Banrao
486,Zhuhai,CN,28.89,few clouds,22.2769,113.5678,Nanyang Seascape Hotel Zhuhai Hotel
519,Sabang,ID,28.40,broken clouds,5.8933,95.3214,Hotel Citra
541,Makakilo City,US,28.00,scattered clouds,21.3469,-158.0858,Marriott's Ko Olina Beach Club
542,Nogales,MX,28.33,clear sky,31.3086,-110.9422,Hotel Marqués De Cima
573,Miri,MY,28.00,few clouds,4.4148,114.0089,"Mega Hotel, Miri"
587,Pochutla,MX,30.00,broken clouds,15.7432,-96.4661,Hotel Posada San Jose
625,Manggar,ID,28.54,broken clouds,-2.8833,108.2667,Guest Hotel
630,Taoudenni,ML,29.25,scattered clouds,22.6783,-3.9836,


In [14]:
# 7. Drop the rows where there is no Hotel Name.
# looks like empty stings are not recognized by dropna so replace with NaN first
# import numpy in first cell
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)

clean_hotel_df = hotel_df.dropna(axis=0, how='any')
clean_hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
11,New Norfolk,AU,29.44,clear sky,-42.7826,147.0587,The Shingles Riverside Cottages
31,Avarua,CK,29.00,broken clouds,-21.2078,-159.7750,Paradise Inn
32,Hobart,AU,29.44,clear sky,-42.8794,147.3294,Mantra on Collins Hobart
40,Vaini,TO,29.00,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
43,Cam Ranh,VN,28.00,broken clouds,11.9214,109.1591,OYO 598 Peony Hotel
44,Carnarvon,AU,29.00,overcast clouds,-24.8667,113.6333,Hospitality Carnarvon
46,Karratha,AU,31.59,clear sky,-20.7377,116.8463,ibis Styles Karratha
48,Hithadhoo,MV,28.79,clear sky,-0.6000,73.0833,Scoop Guest House
62,Mahibadhoo,MV,28.32,broken clouds,3.7833,72.9667,Omadhoo Inn
72,Moratuwa,LK,28.00,few clouds,6.7730,79.8816,Sa Mansion By Yoho


In [26]:
# 8a. Create the output File (CSV)
# clean_hotel_df.isnull().sum()
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv('WeatherPy_vacation.csv', index_label="City_ID")

In [19]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt><b>Hotel Name</b></dt><dd>{Hotel Name}</dd>
<dt><b>City</b></dt><dd>{City}</dd>
<dt><b>Country</b></dt><dd>{Country}</dd>
<dt><b>Description</b></dt><dd>{Description}</dd>
<dt><b>Max Temp</b></dt><dd>{Max Temp} °C</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))